## Momentum Trading Strategy Backtest

**Summary of project:**

### Libraries 

In [4]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import quantstats as qs
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## Importing ETFs 

**Why ETFS?**
> Momentum can look bad on single stocks because a lot of the price move is idiosyncratic noise like one earnings surprise, one headline, one downgrade. An ETF averages many stocks, so that idiosyncratic component diversifies away. What is left is more systematic trend, sector, factor and macro drift, which is exactly what momentum is trying to capture.

In [ ]:
# 1) Define the ETF universe you chose
tickers = ["VTI", "QQQ", "IWM", "VEA", "VWO", "BND", "TLT", "GLD"]

# 2) Download daily adjusted prices
prices = yf.download(
    tickers,
    start="2015-01-01",
    end="2026-01-01",
    interval="1d",
# auto_adjust=True returns prices adjusted for splits and dividends (better for returns)
    auto_adjust=True,     
    progress=False
)["Close"]

# 3) cleaning: sort by date, drop days where everything is missing
prices = prices.sort_index()
prices = prices.dropna(how="all")

# 4) Compute simple daily returns
returns = prices.pct_change().dropna()

# 5) Checks
print("Prices shape:", prices.shape)
print("Returns shape:", returns.shape)
print("\nLast 5 rows of prices:\n", prices.tail())
print("\nLast 5 rows of returns:\n", returns.tail())

Prices shape: (2766, 8)
Returns shape: (2765, 8)

Last 5 rows of prices:
 Ticker            BND         GLD         IWM         QQQ        TLT  \
Date                                                                   
2025-12-24  73.945137  411.929993  252.710007  623.929993  87.694572   
2025-12-26  73.945137  416.739990  251.419998  623.890015  87.405670   
2025-12-29  74.054771  398.600006  249.880005  620.869995  87.734421   
2025-12-30  74.024864  398.890015  248.029999  619.429993  87.525215   
2025-12-31  73.825531  396.309998  246.160004  614.309998  86.827888   

Ticker            VEA         VTI        VWO  
Date                                          
2025-12-24  62.680000  339.880005  53.680000  
2025-12-26  62.810001  339.670013  54.020000  
2025-12-29  62.720001  338.390015  53.610001  
2025-12-30  62.790001  337.850006  53.810001  
2025-12-31  62.470001  335.269989  53.759998  

Last 5 rows of returns:
 Ticker           BND       GLD       IWM       QQQ       TLT      